# 手柄遥控
在本例中，我们将使用连接到web浏览器机器的gamepad控制器来打印手柄的遥控数据。

### 创建手柄控制器
我们要做的第一件事是创建一个' Controller'widget的实例，我们将使用它来驱动我们的USB手柄。
“Controller”小部件接受一个“index”参数，该参数指定控制器的数量。如果你有多个控制器，或者一些游戏手柄以多个控制器的形式出现，这是非常有用的。如果只接入一个手柄，默认index=0，如果有多个则需要根据实际输入对应手柄的编号:
1. 打开[http://html5gamepad.com](http://html5gamepad.com)此网页.  
2. 按下你正在使用的手柄的按键
3. 记住当你按下按键后弹出的相应的索引号

接下来，我们将使用该索引创建并显示控制器。

In [12]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  #用你刚测试过正在使用的控制器的索引号替代
display(controller)
# 如果手柄在休眠状态，请按START键激活连接。

sz_axes = len(controller.axes)
print(f"sz_axes: {sz_axes}")

Controller()

sz_axes: 0


In [3]:
import threading
import time

import inspect
import ctypes

In [4]:
#创建主动停止进程的方法
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)


def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 创建手柄遥杆接收和打印数据的方法  
## 程序功能: 
1. 打印每一次按下的是哪个键。 


In [5]:
def USB_Handle():
    delay_time = 0.01
    while True:
        #因为摇杆手柄个别差异,所有在遥杆复位值不一定是零，所以需要以0.1作为过滤，避免误操作。
        # 打印左边摇杆的数据
        # A1上负下正
        if controller.axes[1].value > 0.1:  # 左边摇杆 向下
            print('L_DOWN=%.2f' % (controller.axes[1].value))
        elif controller.axes[1].value < -0.1:  # 左边摇杆 向上
            print('L_UP=%.2f' % (controller.axes[1].value))
        time.sleep(delay_time)

        # A0左负右正
        if controller.axes[0].value > 0.1:  # 左边摇杆 向右
            print('L_RIGHT=%.2f' % (controller.axes[0].value))
            time.sleep(delay_time)
        elif controller.axes[0].value < -0.1:  # 左边摇杆 向左
            print('L_LEFT=%.2f' % (controller.axes[0].value))
            time.sleep(delay_time)

        # 左边摇杆按下=B10
        if controller.buttons[10].value == True:
            print('L_PRESS')
            time.sleep(delay_time)

        # 打印右边摇杆的模拟值
        # A3上负下正, A2左负右正 
        if controller.axes[3].value < -0.1:
            print('R_UP=%.2f' % (controller.axes[3].value))
            time.sleep(delay_time)
        elif controller.axes[3].value > 0.1:
            print('R_DOWN=%.2f' % (controller.axes[3].value))
            time.sleep(delay_time)

        if controller.axes[2].value > 0.1:
            print('R_RIGHT=%.2f' % (controller.axes[2].value))
            time.sleep(delay_time)
        elif controller.axes[2].value < -0.1:
            print('R_LEFT=%.2f' % (controller.axes[2].value))
            time.sleep(delay_time)

        # 右边摇杆按下=B11
        if controller.buttons[11].value == True:
            print('R_PRESS')
            time.sleep(delay_time)

        # A=B0,B=B1,X=B2,Y=B3
        if controller.buttons[0].value == True:
            print('BTN_A')
            time.sleep(delay_time)
        if controller.buttons[1].value == True:
            print('BTN_B')
            time.sleep(delay_time)
        if controller.buttons[2].value == True:
            print('BTN_X')
            time.sleep(delay_time)
        if controller.buttons[3].value == True:
            print('BTN_Y')
            time.sleep(delay_time)

        # R1=B5,R2=B7
        if controller.buttons[5].value == True:
            print('R1')
            time.sleep(delay_time)
        if controller.buttons[7].value == True:
            print('R2')
            time.sleep(delay_time)

        # L1=B4,L2=B6
        if controller.buttons[4].value == True:
            print('L1')
            time.sleep(delay_time)
        elif controller.buttons[6].value == True:
            print('L2')
            time.sleep(delay_time)

        # SELECT=B8
        if controller.buttons[8].value == True:
            print('SELECT')
            time.sleep(delay_time)
        # START=B9
        if controller.buttons[9].value == True:
            print('START')
            time.sleep(delay_time)

In [7]:
#通过运行下面单元格代码开启手柄的线程
#等待手柄控制线程启动后就可以操作手柄。
thread = threading.Thread(target=USB_Handle, daemon=True)
thread.start()

Exception in thread Thread-6 (USB_Handle):
Traceback (most recent call last):
  File "/opt/user_data/apps/DevOps/anaconda3/envs/robot/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/user_data/apps/DevOps/anaconda3/envs/robot/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/user_data/apps/DevOps/anaconda3/envs/robot/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1839735/473257138.py", line 7, in USB_Handle
IndexError: tuple index out of range


In [15]:
#结束手柄线程。
#如果出现线程启动或者结束失败的情况，
#请重新start一下kernel，再重头一步步运行。
stop_thread(thread)

In [ ]:
USB_Handle()